#MinHash - porównywarka dokumentów

In [ ]:
import re
import os
import time
import numpy as np
import random
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/EMZD/gutenberg
%ls

/content/drive/MyDrive/Colab Notebooks/EMZD/gutenberg
12-0.txt     pg22764.txt  pg5001.txt  pg732.txt
56734-0.txt  pg35688.txt  pg731.txt


In [ ]:
filenames = ['12-0.txt', 'pg22764.txt', 'pg5001.txt', 'pg732.txt', '56734-0.txt', 'pg35688.txt', 'pg731.txt']

#Shingling

In [ ]:
def build_shingles(filename):
    shingles = []
    with open(filename, 'r') as f:
      for line in f:
        line = line.lower()
        line = line.strip()
        line = line.replace(" ", "")
        line = re.sub(r'[^\w\s]','',line)
        for s in range(0, len(line) - 5):
          shingles.append(line[s:s+5]) #bylo k
    return shingles #byly set()

#Porównywarka oparta o index Jaccarda

In [ ]:
def jaccard(set1, set2):
  return len(set1.intersection(set2)) / len(set1.union(set2))

Przedstaw dokumenty jako zbiory shingletów.

In [ ]:
k = 5
shingles = []
for i in range(7):
    shingles.append(build_shingles(filenames[i]))

In [ ]:
print(shingles[0])

['thepr', 'hepro', 'eproj', 'proje', 'rojec', 'oject', 'jectg', 'ectgu', 'ctgut', 'tgute', 'guten', 'utenb', 'tenbe', 'enber', 'nberg', 'berge', 'ergeb', 'rgebo', 'geboo', 'ebook', 'booko', 'ookof', 'okoft', 'kofth', 'ofthr', 'fthro', 'throu', 'hroug', 'rough', 'ought', 'ughth', 'ghthe', 'hthel', 'thelo', 'heloo', 'elook', 'looki', 'ookin', 'oking', 'kingg', 'inggl', 'nggla', 'gglas', 'glass', 'lassb', 'charl', 'harle', 'arles', 'rlesd', 'lesdo', 'esdod', 'sdodg', 'dodgs', 'odgso', 'dgson', 'gsona', 'sonak', 'onaka', 'nakal', 'akale', 'kalew', 'alewi', 'lewis', 'ewisc', 'wisca', 'iscar', 'scarr', 'carro', 'arrol', 'thise', 'hiseb', 'isebo', 'seboo', 'ebook', 'booki', 'ookis', 'okisf', 'kisfo', 'isfor', 'sfort', 'forth', 'orthe', 'rtheu', 'theus', 'heuse', 'euseo', 'useof', 'seofa', 'eofan', 'ofany', 'fanyo', 'anyon', 'nyone', 'yonea', 'onean', 'neany', 'eanyw', 'anywh', 'nywhe', 'ywher', 'where', 'herea', 'ereat', 'reatn', 'eatno', 'atnoc', 'tnoco', 'nocos', 'ocost', 'costa', 'ostan', 

In [ ]:
jaccard(set(shingles[3]), set(shingles[6])) #Podobna tematyka o historii Rzymu

0.9996265082740465

In [ ]:
jaccard(set(shingles[0]), set(shingles[6])) # 0 - Alicja w Krainie Czarów, 6 - Historia Upadku Rzymu

0.15405987281592623

In [ ]:
jaccard(set(shingles[0]), set(shingles[5]) )

0.35411631419939577


#Klasa MinHash

In [ ]:
shingles_array = []
class MyMinHash:
  num_perms = 128
  max_val = 2**32 -1
  perms = [(np.random.randint(0, max_val), np.random.randint(0, max_val))for _ in range (num_perms)]
  minhash_array = []
  file_list = []
  total_shingling_time = 0
  total_hashing_time = 0
  total_minhash_compare_time = 0
  def __init__(self, path):
    self.path = path

  def create_shingle(self, file):
    k = 5
    t0 = time.time()
    shingles = []
    with open(file, 'r') as f:
      for line in f:
        line = line.lower()
        line = line.strip()
        line = line.replace(" ", "")
        line = re.sub(r'[^\w\s]','',line)
        for s in range(0, len(line) - k):
          shingles.append(line[s:s+k]) #bylo k
    print('Shingling %s took: %.2f sec.' % (file, time.time() - t0))
    return shingles #byly set()

  def minhash(self, sequence, indeks, p = 429497311):
    t0 = time.time()
    v = [float('inf') for _ in range(self.num_perms)]
    for s in sequence:
      if not isinstance(s, int):
        s = hash(s)
        for perm_index, perm_parameters in enumerate(self.perms):
          a, b = perm_parameters
          output = (a * s + b) %p
          if v[perm_index] > output:
            v[perm_index] = output
    print('Hashing %s shingles took: %.2f sec.' % (self.file_list[indeks],time.time() - t0))
    return v

  def compare_files_jaccard(self, document):
    global shingles_array
    jaccard_score_array = []
    file_count = len(self.file_list)
    match_percentage = []
    match_index = []
    set1 = []
    helping_index = -1
    for i in range(file_count):
      compare = self.file_list[i]
      if compare == document:
        set1 = shingles_array[i]
        helping_index = i
        break
    if not set1:
        set1 = self.create_shingle(document)
    t0 = time.time()
    for i in range(file_count):
      if i == helping_index:
        pass
      else:
        signature1 = set1
        signature2 = shingles_array[i]
        count = 0
        t1= time.time()
        jaccard_score = len(set(signature1).intersection(set(signature2))) / len(set(signature1).union(set(signature2)))
        print('Jaccard scoring for %s with %s took: %.2f sec.' % (document, self.file_list[i],time.time() - t1))
        match = jaccard_score
        print(match)
        if match > 0.5:
          match_percentage.append(match)
          match_index.append(i)
        print("___________________________________________________")
    print('Comparing all documents using Jaccard Score took: %.2f sec.' % (time.time() - t0))
    print("___________________________________________________")
    print("Documents with at least > %d threshold" % (5))
    for i in range(len(match_index)):
      print("Document %s has %f similiarity with %s" % (document, match_percentage[i], self.file_list[match_index[i]]))
    print("___________________________________________________")

  def compare_files_minhash(self, document):
    match_percentage = []
    match_index = []
    file_list = os.listdir(path)
    file_count = len(file_list)
    for i in range(file_count):
      compare = file_list[i]
      if compare == document:
        hashed_document = self.minhash_array[i]
        helping_index = i
        break
    if not hashed_document:
        shingle_document = self.create_shingle(document)
        hashed_document = self.minhash(shingle_document)
    t0 = time.time()
    for i in range(file_count):
      if i == helping_index:
        pass
      else:
        signature1 = hashed_document
        signature2 = self.minhash_array[i]
        count = 0
        t1= time.time()
        for k in range(0, 128):
          count = count + (signature1[k] == signature2[k])
        print('Comparing %s with %s using MinHash took: %.2f sec.' % (document, self.file_list[i], time.time() - t1))
        match = count / self.num_perms
        print(match)
        if match > 0.5:
          match_percentage.append(match)
          match_index.append(i)
        print("___________________________________________________")
    self.total_minhash_compare_time = time.time() - t0
    print('Comparing all documents with MinHash took: %.2f sec.' % (self.total_minhash_compare_time))
    print("___________________________________________________")
    print("Documents with at least > %d threshold" % (5))
    for i in range(len(match_index)):
      print("Document %s has %f similiarity with %s" % (document, match_percentage[i], self.file_list[match_index[i]]))
    print("___________________________________________________")

  def hash_files(self):
    global shingles_array
    helping_index = -1
    #hashed_document = []
    match_percentage = []
    self.file_list = os.listdir(path)
    file_count = len(self.file_list)
    t0 = time.time()
    for i in range(file_count):
      shingles_array.append(self.create_shingle(self.file_list[i]))
    self.total_shingling_time = time.time() - t0
    print("___________________________________________________")
    print('Shingling all docs took: %.2f sec.' % (self.total_shingling_time))
    print("___________________________________________________")
    t0 = time.time()
    for i in range(file_count):
      self.minhash_array.append(self.minhash(shingles_array[i],i))
    self.total_hashing_time = time.time() - t0
    print("___________________________________________________")
    print('Hashing all shingles took: %.2f sec.' % (self.total_hashing_time))
    print("___________________________________________________")
    print('Preparing files for comparing took: %.2f sec.' % (self.total_hashing_time + self.total_shingling_time))
    print("___________________________________________________")

In [ ]:
path = ("/content/drive/MyDrive/Colab Notebooks/EMZD/gutenberg")
document = "pg731.txt"

In [ ]:
test = MyMinHash(path)

In [ ]:
test.hash_files()

Shingling 56734-0.txt took: 0.29 sec.
Shingling pg22764.txt took: 0.24 sec.
Shingling pg731.txt took: 0.43 sec.
Shingling pg5001.txt took: 0.05 sec.
Shingling pg35688.txt took: 0.03 sec.
Shingling 12-0.txt took: 0.06 sec.
Shingling pg732.txt took: 0.42 sec.
___________________________________________________
Shingling all docs took: 1.52 sec.
___________________________________________________
Hashing 56734-0.txt shingles took: 36.92 sec.
Hashing pg22764.txt shingles took: 32.15 sec.
Hashing pg731.txt shingles took: 53.70 sec.
Hashing pg5001.txt shingles took: 6.53 sec.
Hashing pg35688.txt shingles took: 2.95 sec.
Hashing 12-0.txt shingles took: 5.43 sec.
Hashing pg732.txt shingles took: 53.66 sec.
___________________________________________________
Hashing all shingles took: 191.35 sec.
___________________________________________________
Preparing files for comparing took: 192.87 sec.
___________________________________________________


In [ ]:
test.compare_files_minhash(document)

Comparing pg731.txt with 56734-0.txt using MinHash took: 0.00 sec.
0.34375
___________________________________________________
Comparing pg731.txt with pg22764.txt using MinHash took: 0.00 sec.
0.2734375
___________________________________________________
Comparing pg731.txt with pg5001.txt using MinHash took: 0.00 sec.
0.1328125
___________________________________________________
Comparing pg731.txt with pg35688.txt using MinHash took: 0.00 sec.
0.1015625
___________________________________________________
Comparing pg731.txt with 12-0.txt using MinHash took: 0.00 sec.
0.125
___________________________________________________
Comparing pg731.txt with pg732.txt using MinHash took: 0.00 sec.
1.0
___________________________________________________
Comparing all documents with MinHash took: 0.00 sec.
___________________________________________________
Documents with at least > 5 threshold
Document pg731.txt has 1.000000 similiarity with pg732.txt
__________________________________________

In [ ]:
test.compare_files_jaccard(document)

Jaccard scoring for pg731.txt with 56734-0.txt took: 0.47 sec.
0.34948271398859204
___________________________________________________
Jaccard scoring for pg731.txt with pg22764.txt took: 0.38 sec.
0.29510815532668605
___________________________________________________
Jaccard scoring for pg731.txt with pg5001.txt took: 0.36 sec.
0.1531975729740074
___________________________________________________
Jaccard scoring for pg731.txt with pg35688.txt took: 0.29 sec.
0.1126774211936385
___________________________________________________
Jaccard scoring for pg731.txt with 12-0.txt took: 0.29 sec.
0.15405987281592623
___________________________________________________
Jaccard scoring for pg731.txt with pg732.txt took: 0.59 sec.
0.9996265082740465
___________________________________________________
Comparing all documents using Jaccard Score took: 2.38 sec.
___________________________________________________
Documents with at least > 5 threshold
Document pg731.txt has 0.999627 similiarity with 